# <u><center>Project 2 - Part 7 (Core)
- Authored By: Eric N. Valdez
- Date: 04/19/2024

### For this project you will create a streamlit app to get predictions from your best model.

# Part 1: Preparing Best Models for Streamlit
- ### Create a new Part 7 - Preparing for Streamlit  notebook.

### `In the new notebook`
- ### Define a filpaths ditionary and save it to config/filepaths.json to include file paths for each component you will save(`review below`).
- ### Copy your best models from part 6 into the new notebook.
    - #### Update your code to define the final public-facing class labels. 

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

In [2]:
from pprint import pprint
FPATHS = dict(
    data={
        "raw": {
            "full": "Data-NLP/movie_reviews_v2.csv",  # (This is the original full dataframe we already have)
            "eda": "Data-NLP/movie_reviews_v2.csv" # We haven't saved this yet
        },
        "ml": {
            "train": "Data-NLP/training-data.joblib",  # (X_train,y_train) We haven't saved this yet
            "test": "Data-NLP/testing-data.joblib",  # (X_test,y_test) We haven't saved this yet
        },
    },
    models={
        "gru": "Models/gru/", # We haven't saved this yet
        "random_forest": "Models/random_forest/rf_reg.joblib", # We haven't saved this yet
    },
    images={
        "banner": "Images/IMDB.png", # We haven't saved this yet
    },
)
pprint(FPATHS)

{'data': {'ml': {'test': 'Data-NLP/testing-data.joblib',
                 'train': 'Data-NLP/training-data.joblib'},
          'raw': {'eda': 'Data-NLP/movie_reviews_v2.csv',
                  'full': 'Data-NLP/movie_reviews_v2.csv'}},
 'images': {'banner': 'Images/IMDB.png'},
 'models': {'gru': 'Models/gru/',
            'random_forest': 'Models/random_forest/rf_reg.joblib'}}


In [3]:
# Save the filepaths 
import os, json
os.makedirs('config/', exist_ok=True)
FPATHS_FILE = 'config/filepaths.json'
with open(FPATHS_FILE, 'w') as f:
    json.dump(FPATHS, f)

In [4]:
import os
def create_directories_from_paths(nested_dict):
    """OpenAI. (2023). ChatGPT [Large language model]. https://chat.openai.com 
    Recursively create directories for file paths in a nested dictionary.
    Parameters:
    nested_dict (dict): The nested dictionary containing file paths.
    """
    for key, value in nested_dict.items():
        if isinstance(value, dict):
            # If the value is a dictionary, recurse into it
            create_directories_from_paths(value)
        elif isinstance(value, str):
            # If the value is a string, treat it as a file path and get the directory path
            directory_path = os.path.dirname(value)
            # If the directory path is not empty and the directory does not exist, create it
            if directory_path and not os.path.exists(directory_path):
                os.makedirs(directory_path)
                print(f"Directory created: {directory_path}")

# Use the function on your FPATHS dictionary
create_directories_from_paths(FPATHS)

In [5]:
# We can access a file using our dictionary
FPATHS['data']['raw']['full']

'Data-NLP/movie_reviews_v2.csv'

In [6]:
# We can access a file using our dictionary
FPATHS['models']['random_forest']

'Models/random_forest/rf_reg.joblib'

In [7]:
# Confirm the images is in the correct location
from IPython.display import display, Markdown
Markdown(f"<img src='{FPATHS['images']['banner']}'>")

<img src='Images/IMDB.png'>

## `Saving Your Models`
- ### For your Machine Learning Model:
    - Save your training data ([X_train, y_train])
    - Save your test data ([X_test, y_test])
    - Save your target_lookup dictionary and/or your label encoder
    - Save your best model

- ### For your  Deep NLP model:
    - Save your training data (train_ds)
    - Save your test data (test_ds)
    - Save your best neural network.
        - `Reminder:`use safe_format = 'tf' to save the model in a folder of repo-friendly files.

# Part 2: Streamlit App

### You will create a Streamlit app to get model predictions for user-entered text. 

### You may select `either your best machine learning model or deep nlp model.` **Note:** for portfolio purposes, it would be best to eventually create an app for both.
- #### Create a Stramlit app for getting predictions for a user-entered text from your loaded model.
- #### (Optional but recommended); Include a Lime Text Explainer explanation for the prediction.
- #### Include the ability to load the training and test data to evaluate the model.